In [56]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import functools as ft
import numpy as np
from sklearn.model_selection import KFold
from sklearn import datasets
import random
from sklearn.model_selection import ShuffleSplit
from sklearn.naive_bayes import GaussianNB

# 0 Data processing

In [ ]:
#!mkdir ../data/iris
#!mv /Users/Christophe/Downloads/IRIS.csv ../data/iris/iris.csv

In [ ]:
#!wc -l ../data/iris/iris.csv

In [ ]:
#iris_df = pd.read_csv("../data/iris/iris.csv", header= None)

In [ ]:
#iris_df.columns = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'y']

In [ ]:
#iris_df.head(3)

In [ ]:
#iris_df = shuffle(iris_df)

In [ ]:
#def f(x) :
#     if x == "setosa":
#         return(0)
#     elif x == "virginica":
#         return(1)
#     else: 
#         return(3)

In [ ]:
# iris_df['y'] = list(map(f, iris_df['y']))

In [2]:
# set random generator seed
random.seed(20170312)
index = shuffle(range(0,149))
index

[34,
 147,
 138,
 79,
 143,
 44,
 74,
 13,
 27,
 29,
 28,
 119,
 55,
 60,
 135,
 45,
 64,
 53,
 33,
 134,
 38,
 93,
 120,
 130,
 86,
 144,
 82,
 118,
 99,
 141,
 3,
 0,
 136,
 105,
 30,
 129,
 32,
 127,
 77,
 94,
 111,
 36,
 69,
 5,
 63,
 103,
 140,
 54,
 10,
 50,
 26,
 107,
 96,
 117,
 100,
 83,
 15,
 95,
 58,
 4,
 148,
 65,
 25,
 114,
 92,
 48,
 115,
 110,
 57,
 51,
 16,
 49,
 2,
 125,
 35,
 68,
 102,
 17,
 1,
 70,
 98,
 7,
 8,
 21,
 66,
 46,
 71,
 40,
 122,
 22,
 80,
 90,
 97,
 52,
 104,
 76,
 113,
 56,
 39,
 145,
 41,
 131,
 142,
 9,
 116,
 37,
 101,
 23,
 91,
 89,
 59,
 87,
 78,
 109,
 14,
 133,
 88,
 106,
 12,
 67,
 84,
 47,
 31,
 128,
 123,
 112,
 72,
 85,
 73,
 132,
 24,
 146,
 81,
 43,
 6,
 108,
 139,
 42,
 20,
 62,
 121,
 11,
 18,
 137,
 126,
 19,
 61,
 124,
 75]

In [3]:
iris_df = datasets.load_iris()
len(iris_df.data)

150

# 1 Split data into train & test

In [ ]:
#train = iris_df.head(70)

In [ ]:
#train.head(2)

In [4]:
#x_train = train[['x1' ,'x2','x3' ,'x4' ,'x5' ,'x6' ,'x7', 'x8']]
x_train = iris_df.data[index[:100], ]

In [ ]:
#x_train = x_train.apply(list, 1)

In [ ]:
#x_train

In [5]:
#y_train = train['y']
y_train = iris_df.target[index[:100]]

In [ ]:
#test = iris_df.tail(30)

In [6]:
#x_test = test[['x1' ,'x2','x3' ,'x4' ,'x5' ,'x6' ,'x7', 'x8']]
x_test = iris_df.data[index[100:150],]

In [7]:
#y_test = test['y']
y_test = iris_df.target[index[100:150]]

In [8]:
clf = LogisticRegression(solver='newton-cg', max_iter=100, random_state=42,
                             multi_class='multinomial').fit(x_train, y_train)

In [9]:
clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

## 1.1 perf on train

In [10]:
clf.score(x_train, y_train)

0.96999999999999997

In [12]:
y_pred = clf.predict(x_test)

## 1.2 perf on test

In [13]:
clf.score(x_test, y_test)

0.91836734693877553

In [14]:
y_test

array([0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 0, 0,
       2, 2, 2, 1, 1, 1, 2, 0, 2, 1, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 2, 2, 0,
       1, 2, 1])

In [15]:
y_pred

array([0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       2, 1, 2, 1, 1, 1, 2, 0, 2, 1, 0, 0, 2, 2, 0, 0, 1, 2, 0, 0, 2, 1, 0,
       1, 2, 1])

# 2 Cross-validations

In [59]:
apps = pd.DataFrame(0., index=np.arange(10), columns=["logreg", "svm", "NB"])
tests = pd.DataFrame(0., index=np.arange(10), columns=["logreg", "svm", "NB"])

In [61]:
ss = ShuffleSplit(n_splits=10, random_state=0)
i = 0
for train_index, test_index in ss.split(iris_df.target):
    logreg_clf = LogisticRegression(solver='newton-cg', max_iter=100, random_state=42,
                             multi_class='multinomial').fit(iris_df.data[train_index,], iris_df.target[train_index])
    svm_clf = svm.SVC(decision_function_shape='ovo').fit(iris_df.data[train_index,], iris_df.target[train_index])
    nb_clf = GaussianNB().fit(iris_df.data[train_index,], iris_df.target[train_index])
    apps["logreg"][i] = logreg_clf.score(iris_df.data[train_index,], iris_df.target[train_index])
    tests["logreg"][i] = logreg_clf.score(iris_df.data[test_index], iris_df.target[test_index])
    apps["svm"][i] = svm_clf.score(iris_df.data[train_index,], iris_df.target[train_index])
    tests["svm"][i] = svm_clf.score(iris_df.data[test_index], iris_df.target[test_index])
    apps["NB"][i] = nb_clf.score(iris_df.data[train_index,], iris_df.target[train_index])
    tests["NB"][i] = nb_clf.score(iris_df.data[test_index], iris_df.target[test_index])
    i = i + 1

In [69]:
apps = 1. - apps
apps

,logreg,svm,NB
0,0.029630,0.014815,0.044444
1,0.022222,0.014815,0.029630
2,0.022222,0.014815,0.037037
3,0.022222,0.014815,0.044444
4,0.029630,0.014815,0.044444
5,0.029630,0.022222,0.029630
6,0.029630,0.014815,0.044444
7,0.029630,0.014815,0.037037
8,0.029630,0.022222,0.044444
9,0.029630,0.014815,0.037037


In [70]:
# mean training error => first thought: svm is the best model 
np.mean(apps)

logreg    0.027407
svm       0.016296
NB        0.039259
dtype: float64

In [71]:
# std training error
np.std(apps)

logreg    0.003395
svm       0.002963
NB        0.005785
dtype: float64

In [72]:
# rule of thumb : minimize the average error + 1 std => svm is still the best model
np.mean(apps) + np.std(apps)

logreg    0.030802
svm       0.019259
NB        0.045045
dtype: float64

In [ ]:
# Naive Bayes works not very well because of strong theorical hypotheses (features independence)
# which is not intuitively met in this case.
# SVM work better than logistic regression because of the kernel transformation